In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys

sys.path.append("../../")

import numpy as np
import pandas as pd

In [ ]:
from choice_learn.datasets import load_swissmetro
from choice_learn.data import ChoiceDataset
swiss_df = load_swissmetro(as_frame=True)
swiss_df.head()

In [ ]:
test_df = pd.read_csv("https://raw.githubusercontent.com/BSifringer/EnhancedDCM/refs/heads/master/ready_example/swissmetro_paper/swissmetro_test.dat", sep="\t")
train_df = pd.read_csv("https://raw.githubusercontent.com/BSifringer/EnhancedDCM/refs/heads/master/ready_example/swissmetro_paper/swissmetro_train.dat", sep="\t")

In [ ]:
test_df.head()

In [ ]:
train_df.head()

In [ ]:
swiss_df = swiss_df.loc[swiss_df.CAR_AV == 1]
swiss_df = swiss_df.loc[swiss_df.SM_AV == 1]
swiss_df = swiss_df.loc[swiss_df.TRAIN_AV == 1]

test_df = test_df.loc[test_df.CAR_AV == 1]
test_df = test_df.loc[test_df.SM_AV == 1]
test_df = test_df.loc[test_df.TRAIN_AV == 1]

train_df = train_df.loc[train_df.CAR_AV == 1]
train_df = train_df.loc[train_df.SM_AV == 1]
train_df = train_df.loc[train_df.TRAIN_AV == 1]

In [ ]:
# swiss_df = swiss_df.loc[swiss_df.PURPOSE.isin([1, 3])]

# Normalizing values by 100
swiss_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] = (
    swiss_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] / 100.0
)

swiss_df[["TRAIN_HE", "SM_HE"]] = (
    swiss_df[["TRAIN_HE", "SM_HE"]] / 100.0
)

swiss_df["train_free_ticket"] = swiss_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)
swiss_df["sm_free_ticket"] = swiss_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)

swiss_df["TRAIN_travel_cost"] = swiss_df.apply(
    lambda row: (row["TRAIN_CO"] * (1 - row["train_free_ticket"])) / 100, axis=1
)
swiss_df["SM_travel_cost"] = swiss_df.apply(
    lambda row: (row["SM_CO"] * (1 - row["sm_free_ticket"])) / 100, axis=1
)
swiss_df["CAR_travel_cost"] = swiss_df.apply(lambda row: row["CAR_CO"] / 100, axis=1)


####

# Normalizing values by 100
train_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] = (
    train_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] / 100.0
)

train_df[["TRAIN_HE", "SM_HE"]] = (
    train_df[["TRAIN_HE", "SM_HE"]] / 100.0
)

train_df["train_free_ticket"] = train_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)
train_df["sm_free_ticket"] = train_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)

train_df["TRAIN_travel_cost"] = train_df.apply(
    lambda row: (row["TRAIN_CO"] * (1 - row["train_free_ticket"])) / 100, axis=1
)
train_df["SM_travel_cost"] = train_df.apply(
    lambda row: (row["SM_CO"] * (1 - row["sm_free_ticket"])) / 100, axis=1
)
train_df["CAR_travel_cost"] = train_df.apply(lambda row: row["CAR_CO"] / 100, axis=1)


# Normalizing values by 100
test_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] = (
    test_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] / 100.0
)

test_df[["TRAIN_HE", "SM_HE"]] = (
    test_df[["TRAIN_HE", "SM_HE"]] / 100.0
)

test_df["train_free_ticket"] = test_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)
test_df["sm_free_ticket"] = test_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)

test_df["TRAIN_travel_cost"] = test_df.apply(
    lambda row: (row["TRAIN_CO"] * (1 - row["train_free_ticket"])) / 100, axis=1
)
test_df["SM_travel_cost"] = test_df.apply(
    lambda row: (row["SM_CO"] * (1 - row["sm_free_ticket"])) / 100, axis=1
)
test_df["CAR_travel_cost"] = test_df.apply(lambda row: row["CAR_CO"] / 100, axis=1)



In [ ]:
swiss_df.SM_SEATS = swiss_df.SM_SEATS.astype("float32")
train_df.SM_SEATS = train_df.SM_SEATS.astype("float32")
test_df.SM_SEATS = test_df.SM_SEATS.astype("float32")

train_df.CHOICE = train_df.CHOICE - 1
test_df.CHOICE = test_df.CHOICE - 1

In [ ]:
dataset = ChoiceDataset.from_single_wide_df(df=swiss_df, choices_column="CHOICE", items_id=["TRAIN", "SM", "CAR"],
shared_features_columns=["GA", "AGE", "SM_SEATS", "LUGGAGE", "SM_SEATS", 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'MALE', 'INCOME', 'ORIGIN', 'DEST'],
items_features_suffixes=["TT", "travel_cost", "HE"], choice_format="items_index")

train_dataset = ChoiceDataset.from_single_wide_df(df=train_df, choices_column="CHOICE", items_id=["TRAIN", "SM", "CAR"],
shared_features_columns=["GA", "AGE", "SM_SEATS", "LUGGAGE", "SM_SEATS", 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'MALE', 'INCOME', 'ORIGIN', 'DEST'],
items_features_suffixes=["TT", "travel_cost", "HE"], choice_format="items_index")

test_dataset = ChoiceDataset.from_single_wide_df(df=test_df, choices_column="CHOICE", items_id=["TRAIN", "SM", "CAR"],
shared_features_columns=["GA", "AGE", "SM_SEATS", "LUGGAGE", "SM_SEATS", 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'MALE', 'INCOME', 'ORIGIN', 'DEST'],
items_features_suffixes=["TT", "travel_cost", "HE"], choice_format="items_index")

In [ ]:
print(len(dataset), len(train_dataset), len(test_dataset))

In [ ]:
from choice_learn.models import ConditionalLogit

clogit = ConditionalLogit()
clogit.add_shared_coefficient(feature_name="TT", coefficient_name="beta_time",  items_indexes=[0, 1, 2])
clogit.add_shared_coefficient(feature_name="travel_cost", coefficient_name="beta_cost", items_indexes=[0, 1, 2])
clogit.add_shared_coefficient(feature_name="HE",  coefficient_name="beta_freq",items_indexes=[0, 1])
clogit.add_shared_coefficient(feature_name="GA",  coefficient_name="beta_GA",items_indexes=[0, 1])
clogit.add_shared_coefficient(feature_name="AGE", coefficient_name="beta_age", items_indexes=[0])
clogit.add_shared_coefficient(feature_name="LUGGAGE", coefficient_name="beta_luggage", items_indexes=[2])
clogit.add_shared_coefficient(feature_name="SM_SEATS", coefficient_name="beta_seats", items_indexes=[1])
clogit.add_coefficients(feature_name="intercept", items_indexes=[1, 2])

clogit.fit(train_dataset)

In [ ]:
clogit.trainable_weights

In [ ]:
clogit.evaluate(train_dataset) * len(train_dataset), clogit.evaluate(test_dataset) * len(test_dataset)

In [ ]:
indexes = np.random.permutation(len(dataset))
train = indexes[:7234]
test = indexes[7234:]

In [ ]:
from choice_learn.models.learning_mnl import LearningMNL

swiss_model = LearningMNL(optimizer="Adam", 
nn_features=['PURPOSE', 'FIRST', 'TICKET', 'WHO', 'MALE', 'INCOME', 'ORIGIN', 'DEST'], nn_layers_widths=[100], epochs=500, batch_size=32)
swiss_model.add_shared_coefficient(feature_name="TT", items_indexes=[0, 1, 2])
swiss_model.add_shared_coefficient(feature_name="travel_cost", items_indexes=[0, 1, 2])
swiss_model.add_shared_coefficient(feature_name="HE", items_indexes=[0, 1])
swiss_model.add_shared_coefficient(feature_name="GA", items_indexes=[0, 1])
swiss_model.add_shared_coefficient(feature_name="AGE", items_indexes=[0])
swiss_model.add_shared_coefficient(feature_name="LUGGAGE", items_indexes=[2])
swiss_model.add_shared_coefficient(feature_name="SM_SEATS", items_indexes=[1])
swiss_model.add_coefficients(feature_name="intercept", items_indexes=[1, 2])

hist = swiss_model.fit(train_dataset, val_dataset=test_dataset, verbose=1)

In [ ]:

hist2 = swiss_model.fit(train_dataset, val_dataset=test_dataset, verbose=1)

In [ ]:

hist2 = swiss_model.fit(train_dataset, val_dataset=test_dataset, verbose=1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(hist["train_loss"])
plt.plot(hist["test_loss"])
plt.subplot(1, 2, 2)
plt.plot(hist2["train_loss"])
plt.plot(hist2["test_loss"])

In [ ]:
 swiss_model.evaluate(dataset[:7234], batch_size=32) * 7234

In [ ]:
swiss_model.evaluate(dataset[7234:], batch_size=32) * 1802

In [ ]:
swiss_model.trainable_weights

In [ ]:
len(dataset)

In [ ]:
swiss_model.nn_model.summary()